In [17]:
import os
import torch

from modeling.modeling_llada import LLaDAModelLM
from transformers import AutoTokenizer
from get_log_likelihood import forward_process, get_log_likelihood
from datasets import load_dataset
from jinyu_tokenizer import Tokenizer_
from jinyu_inspect import jinyu_inspect_file

In [2]:
id_model = 'GSAI-ML/LLaDA-8B-Base'
path_cache_base = os.environ['HF_HUB_CACHE']
folder_model = '--'.join(['models'] + id_model.split('/'))
path_cache_model = os.path.join(path_cache_base, folder_model)
path_snapshot_model = os.path.join(path_cache_model, 'snapshots')
folder_snapshot_model_1 = [entity for entity in os.listdir(path_snapshot_model) if entity[0] != '.'][0]
path_snapshot_model_1 = os.path.join(path_snapshot_model, folder_snapshot_model_1)
print(path_snapshot_model_1)


/home/exx/hf_hub_cache/models--GSAI-ML--LLaDA-8B-Base/snapshots/0f2787f2d87eac5eed8a087d5ecd24277e6255b2


In [3]:
'''load tokenizer'''
tokenizer = AutoTokenizer.from_pretrained(
    path_snapshot_model_1,
    local_files_only=True,
    trust_remote_code=True
)

In [ ]:
'''load model'''

model_kwargs = {}
model = LLaDAModelLM.from_pretrained(
    path_snapshot_model_1,
    local_files_only=True,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    **model_kwargs
)

model = model.eval()
device_for_input = model.get_input_embeddings().weight.device

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 6/6 [00:02<00:00,  2.85it/s]


In [ ]:
names_dataset = [('Idavidrein/gpqa', 'gpqa_main'), ('Salesforce/wikitext', 'wikitext-2-raw-v1')]
ds = load_dataset(*names_dataset[1], split='test')['text']
# ds = [data for data in ds['text'] if len(data)]
# print(ds)


In [28]:
ds[:200]

{'text': ['',
  ' = Robert Boulter = \n',
  '',
  ' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n',
  ' In 2006 , Boulter starred alongside Whishaw in the play Citizenship written by Ma

In [ ]:
# class Tokenizer_test(Tokenizer_):
#     def _tokenize(self, e):
#         prefix, target = self._encode_pair(e['prefix'], e['target'])
#         return {
#             'prefix_text': e['prefix'],
#             'target_text': e['target'],
#             'prefix': prefix,
#             'target': target
#         }
#     # end
# # end


ds = ds.map(type("Tokenizer_test", (Tokenizer_,), {
    "_tokenize": lambda self, t: self._encode_pair(t['prefix'], t['target'])
})(tokenizer))

# ds = ds.map(Tokenizer_test(tokenizer))
ds = ds.with_format("torch")
prompt_len = [len(x["prefix"]) + len(x["target"]) for x in ds]
assert max(prompt_len) <= 4096

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

KeyError: 'prefix'

In [ ]:

out = []
with torch.no_grad():
    for elem in tqdm(ds, desc="Computing likelihood..."):
        prefix = elem["prefix"]
        target = elem["target"]

        ll = get_loglikelihood(prefix, target)
        out.append(ll)
    # end




# # this prompt and answer is from Hellaswag dataset
# prompt = 'Roof shingle removal: A man is sitting on a roof. He'
# answer = ' is using wrap to wrap a pair of skis.'
# for i in range(10):
#     ids_prompt = torch.tensor(tokenizer(prompt)['input_ids']).to(device_for_input)
#     ids_answer = torch.tensor(tokenizer(answer)['input_ids']).to(device_for_input)
#     print(get_log_likelihood(model, ids_prompt, ids_answer, mc_num=128))
# # end